# Posizionamento con altri algoritmi
- 1. Random
- 2. DBSCAN

In [1]:
import pandas as pd
import shapely
#import pyproj
#from pyproj import CRS
from shapely.geometry import Point, Polygon
import geopandas
import matplotlib
#from planar import BoundingBox
#from geopy import distance 
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.interpolate import griddata
import numpy as np
#import planar
import yaml
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
import cartopy.geodesic as GD


with open('../conf.yaml') as f:
    conf = yaml.load(f, Loader=yaml.FullLoader)
path = conf["path"]
Dr = conf["radius"]
Cr = conf["soglia"]
R0_iterations = conf["R0_iterations"]
pathAreeProibite = conf["pathAreeProibite"]
pathDFLow = conf["pathDFLow"]
pathDFNormal = conf["pathDFNormal"]
pathDFHigh = conf["pathDFHigh"]
pathDFBorder = conf ["pathDFBorder"]
pathDFFull = conf ["pathDFFull"]
pathBox = conf["pathBox"]

#lettura dei df aumentati
# Low
b = geopandas.read_file(pathDFLow+"DFLow.geojson",index_col = 0,columns=['probability','geometry'])
dfLow=geopandas.GeoDataFrame(b,crs="EPSG:4326")
dfLowDiscovered = dfLow[(dfLow.probability<=Cr)]

# Normal
b = geopandas.read_file(pathDFNormal+"DFNormal.geojson",index_col = 0,columns=['probability','geometry'])
dfNormal = geopandas.GeoDataFrame(b,crs="EPSG:4326")
dfNormalDiscovered = dfNormal[(dfNormal.probability<=Cr)]


# High
b = geopandas.read_file(pathDFHigh+"DFHigh.geojson",index_col = 0,columns=['probability','geometry'])
dfHigh=geopandas.GeoDataFrame(b,crs="EPSG:4326")
dfHighDiscovered = dfHigh[(dfHigh.probability<=Cr)]

# Border
b = geopandas.read_file(pathDFBorder+"DFBorder.geojson",index_col = 0,columns=['probability','geometry'])
dfBorder=geopandas.GeoDataFrame(b,crs="EPSG:4326")
dfBorderDiscovered = dfBorder[(dfBorder.probability<=Cr)]


#prelevo i box per il test
Box = geopandas.GeoDataFrame(geopandas.read_file(pathBox),crs="EPSG:4326")
Box=Box.set_index('index')
boxLow=Box.at["boxLow",'geometry']
boxNormal=Box.at["boxNormal",'geometry']
boxHigh=Box.at["boxHigh",'geometry']
boxBorder=Box.at["boxBorder",'geometry']


#Prelevo le aree proibite
areeProibite = geopandas.GeoDataFrame(geopandas.read_file(pathAreeProibite),crs = "EPSG:4326")
areeProibite=areeProibite['geometry']


# 1. Random Deployement

# Funzione per il posizionamento random di una stazione

In [2]:

def randomTest1(dfPoint,dfPointDiscovered,box) :

    boundsBox = box.bounds
    minlat=boundsBox[1]
    maxlat=boundsBox[3]
    minlon=boundsBox[0]
    maxlon=boundsBox[2]
    mat = np.zeros((R0_iterations,2))

    #genero lon e lat casuali delle stazioni
    for k in range(R0_iterations) : 
        lon = np.random.uniform (minlon,maxlon,1)
        mat[k][0] = lon
        lat = np.random.uniform (minlat,maxlat,1)
        mat [k][1] = lat
        
    #costruisco un geodataframe con le posizioni generate casualmente
    dfStations = geopandas.GeoDataFrame(crs="EPSG:4326", geometry=geopandas.points_from_xy(mat[:,0], mat[:,1]))
    dfStations['pointsCoveredUAV'] = ""
    dfStations['pointsCoveredTotal'] = ""
    
    #conteggio i punti coperti dalle varie stazioni
    for row in dfStations.itertuples():
        lon = row.geometry.x
        lat = row.geometry.y
        circle_points = GD.Geodesic().circle( lon= lon, lat=lat, radius = Dr)
        circle_poly = Polygon(circle_points)
        dfCircle = geopandas.GeoDataFrame({'geometry':[circle_poly]},crs="EPSG:4326")
        df = geopandas.sjoin(dfPointDiscovered, dfCircle, how = "inner", op="intersects")
        dfStations.at[row.Index,'pointsCoveredUAV'] = df.shape[0]
        df = geopandas.sjoin(dfPoint, dfCircle, how = "inner", op="intersects")
        dfStations.at[row.Index,'pointsCoveredTotal'] = df.shape[0]
    
    print(dfStations.head())
    return dfStations


# Funzione per il posizionamento random di 3 stazioni

In [4]:
import cartopy.geodesic as GD

def randomTest3(dfPoint,dfPointDiscovered,box) :

    boundsBox = box.bounds
    minlat=boundsBox[1]
    maxlat=boundsBox[3]
    minlon=boundsBox[0]
    maxlon=boundsBox[2]
    mat = np.zeros((R0_iterations,6))

    #genero lon e lat casuali delle stazioni
    for k in range(R0_iterations) : 
        for i in range (3) :
            lon = np.random.uniform (minlon,maxlon,1)
            mat[k][0+i*2] = lon
            lat = np.random.uniform (minlat,maxlat,1)
            mat [k][1+i*2] = lat

    #costruisco un geodataframe con le posizioni generate casualmente
    dfStations = pd.DataFrame(mat,columns=['lon1','lat1','lon2','lat2','lon3','lat3'])
    dfStations['pointsCoveredUAV'] = ""
    dfStations['pointsCoveredTotal'] = ""

    #conteggio i punti coperti dalle varie stazioni
    for row in dfStations.itertuples():
        circle_poly1 = Polygon (GD.Geodesic().circle( lon= row.lon1, lat=row.lat1, radius = Dr))
        circle_poly2 = Polygon(GD.Geodesic().circle( lon= row.lon2, lat=row.lat2, radius = Dr))
        circle_poly3 = Polygon(GD.Geodesic().circle( lon= row.lon3, lat=row.lat3, radius = Dr))
        dfCircle = geopandas.GeoDataFrame({'geometry':[circle_poly1,circle_poly2,circle_poly3]},crs="EPSG:4326")
        
        df = geopandas.sjoin(dfPointDiscovered, dfCircle, how = "inner", op="intersects")
        df.drop_duplicates(subset=['geometry'],inplace = True)
        dfStations.at[row.Index,'pointsCoveredUAV'] = df.shape[0]
        df = geopandas.sjoin(dfPoint, dfCircle, how = "inner", op="intersects")
        df.drop_duplicates(subset=['geometry'],inplace = True)
        dfStations.at[row.Index,'pointsCoveredTotal'] = df.shape[0]
    print(dfStations.head())
    return dfStations

# Funzione per il posizionamento random di 3 stazioni con zone non ammesse

In [8]:
def randomTest3_areeProibite(dfPoint,dfPointDiscovered,box) :

    boundsBox = box.bounds
    minlat=boundsBox[1]
    maxlat=boundsBox[3]
    minlon=boundsBox[0]
    maxlon=boundsBox[2]
    mat = np.zeros((R0_iterations,6))

    #genero lon e lat casuali delle stazioni e controllo che siano valide
    for k in range(R0_iterations) : 
        i = 0
        while i<3 :
            lon = np.random.uniform (minlon,maxlon,1)
            lat = np.random.uniform (minlat,maxlat,1)
            if invalidCenter((lon,lat),areeProibite) == False:
                mat[k][0+i*2] = lon
                mat [k][1+i*2] = lat
                i = i+1

    #costruisco un dataframe con le posizioni generate casualmente
    dfStations = pd.DataFrame(mat,columns=['lon1','lat1','lon2','lat2','lon3','lat3'])
    dfStations['pointsCoveredUAV'] = ""
    dfStations['pointsCoveredTotal'] = ""

    #conteggio i punti coperti dalle varie stazioni
    for row in dfStations.itertuples():
        circle_poly1 = Polygon (GD.Geodesic().circle( lon= row.lon1, lat=row.lat1, radius = Dr))
        circle_poly2 = Polygon(GD.Geodesic().circle( lon= row.lon2, lat=row.lat2, radius = Dr))
        circle_poly3 = Polygon(GD.Geodesic().circle( lon= row.lon3, lat=row.lat3, radius = Dr))
        dfCircle = geopandas.GeoDataFrame({'geometry':[circle_poly1,circle_poly2,circle_poly3]},crs="EPSG:4326")
        
        df = geopandas.sjoin(dfPointDiscovered, dfCircle, how = "inner", op="intersects")
        df.drop_duplicates(subset=['geometry'],inplace = True)
        dfStations.at[row.Index,'pointsCoveredUAV'] = df.shape[0]
        df = geopandas.sjoin(dfPoint, dfCircle, how = "inner", op="intersects")
        df.drop_duplicates(subset=['geometry'],inplace = True)
        dfStations.at[row.Index,'pointsCoveredTotal'] = df.shape[0]
    print(dfStations.head())
    return dfStations

# Calcolo di dati statistici e scrittura su file

In [3]:

def statistics(dfStations,path,k) :

    pointsCoveredUAV = dfStations['pointsCoveredUAV']
    pointsCoveredTotal = dfStations['pointsCoveredTotal']
    meanCoverageUAV = pointsCoveredUAV.mean(axis=0)
    stdCoverageUAV = pointsCoveredUAV.std()
    percentile_75UAV = pointsCoveredUAV.quantile(.75)
    percentile_25UAV = pointsCoveredUAV.quantile(.25)
    maxUAV = pointsCoveredUAV.max()
    meanCoverageTot = pointsCoveredTotal.mean(axis=0)
    percentile_75Tot = pointsCoveredTotal.quantile(.75)
    percentile_25Tot = pointsCoveredTotal.quantile(.25)
    maxTot = pointsCoveredTotal.max()
    toWrite = "mean Coverage UAV = " + str(meanCoverageUAV) +"std Coverage UAV ="+str(stdCoverageUAV)+ " 75 Percentile UAV = " + str(percentile_75UAV) + " 25 Percentile UAV= " + str(percentile_25UAV) + " maxUAV="+str(maxUAV)+"\n mean Coverage Tot = " + str(meanCoverageTot) + " 75 Percentile Tot = " + str(percentile_75Tot) + " 25 Percentile Tot= " + str(percentile_25Tot) +" maxTot = " + str(maxTot)
    file = open(path+"randomTest"+str(k) + ".odt","w")
    file.write(toWrite)
    file.close()

    

# Test 
## posizionamento random di 1 stazione in 3 scenari di coverage

In [4]:
#dfStationsLow = randomTest1(dfLow,dfLowDiscovered,boxLow)
#dfStationsNormal = randomTest1(dfNormal,dfNormalDiscovered,boxNormal)
dfStationsHigh = randomTest1(dfHigh,dfHighDiscovered,boxHigh)

#statistics (dfStationsLow,pathDFLow,1)
#statistics(dfStationsNormal,path  DFNormal,1)
statistics(dfStationsHigh,pathDFHigh,1)

                     geometry pointsCoveredUAV pointsCoveredTotal
0  POINT (116.30519 39.92838)                3                  5
1  POINT (116.39083 39.95337)                1                  2
2  POINT (116.38650 39.93664)                0                  2
3  POINT (116.39993 39.97729)                2                  3
4  POINT (116.37232 39.91677)                0                  1


# Test 
## posizionamento random di 3 stazioni in 3 scenari di coverage

In [6]:
dfStationsLow = randomTest3(dfLow,dfLowDiscovered,boxLow)
dfStationsNormal = randomTest3(dfNormal,dfNormalDiscovered,boxNormal)
dfStationsHigh = randomTest3(dfHigh,dfHighDiscovered,boxHigh)

statistics (dfStationsLow,pathDFLow,3)
statistics(dfStationsNormal,pathDFNormal,3)
statistics(dfStationsHigh,pathDFHigh,3)

         lon1       lat1        lon2       lat2        lon3       lat3  \
0  116.092816  40.368927  116.078167  40.383442  116.060356  40.326075   
1  116.037462  40.335508  116.073057  40.393121  116.076211  40.310233   
2  116.004283  40.355019  116.038178  40.359181  116.061761  40.370049   
3  116.087696  40.379555  116.098453  40.357273  116.055449  40.308994   
4  116.057912  40.329991  116.066823  40.363288  116.084295  40.368408   

  pointsCoveredUAV pointsCoveredTotal  
0                3                  3  
1                3                  3  
2                6                  6  
3                3                  3  
4                7                  7  
         lon1       lat1        lon2       lat2        lon3       lat3  \
0  116.632500  40.345734  116.640726  40.295931  116.671125  40.296378   
1  116.668813  40.291339  116.664603  40.295602  116.667503  40.335178   
2  116.653420  40.292112  116.610462  40.328481  116.694396  40.340200   
3  116.630953  40.3

# Test
## posizionamento random di 3 stazioni + zone non ammesse

In [9]:
dfStationsLow = randomTest3_areeProibite(dfLow,dfLowDiscovered,boxLow)
dfStationsNormal = randomTest3_areeProibite(dfNormal,dfNormalDiscovered,boxNormal)
dfStationsHigh = randomTest3_areeProibite(dfHigh,dfHighDiscovered,boxHigh)

statistics (dfStationsLow,pathDFLow,3.1)
statistics(dfStationsNormal,pathDFNormal,3.1)
statistics(dfStationsHigh,pathDFHigh,3.1)

         lon1       lat1        lon2       lat2        lon3       lat3  \
0  116.025164  40.318249  116.096371  40.306841  116.008612  40.306183   
1  116.015861  40.399126  116.031366  40.331673  116.003883  40.307731   
2  116.076364  40.341505  116.001304  40.362794  116.060855  40.342187   
3  116.014565  40.391072  116.086505  40.373373  116.005455  40.332891   
4  116.028303  40.318388  116.018779  40.344024  116.094133  40.354977   

  pointsCoveredUAV pointsCoveredTotal  
0                7                  7  
1                8                  8  
2                7                  7  
3                8                  8  
4                5                  5  
         lon1       lat1        lon2       lat2        lon3       lat3  \
0  116.625333  40.279002  116.670956  40.311074  116.643759  40.344979   
1  116.678640  40.280816  116.655916  40.309669  116.680910  40.261541   
2  116.630137  40.344000  116.638256  40.299401  116.654807  40.294787   
3  116.646812  40.3

# 2. DBSCAN Deployment